## My game of life

Ok, it's been a while since I wanted to implement a game fo life, but lack of time and motivation were against me.
I have both right now so ... let's do this a pythonic way !

### What is the game of life ?

It is a cellular automation, designed by Conway in 1970
More info here : https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

### Rules of the game :
- A cell can be either dead or alive
- Any live cell with two or three neighbors survives
- Any dead cell with three live neighbors becomes a live cell
- All other live cells die in the next generation. Similarly, all other dead cells stay dead

In [75]:
import tkinter as tk

class Grid():
    '''
    This class implement the grid that contains all the cells in the game
    '''
    def __init__(self, canvas, width, height, cellsize):
        self.cellsize = cellsize
        self.canvas = canvas    

        # Variable calculated from default ones
        self.nbr_cell_width = int(width / cellsize)
        self.nbr_cell_height = int(height / cellsize)

        # List of all cells in the grid
        self.cells = []
        
    def generate_grid(self):
        '''
        Generate aa grid with empty rectangle
        Also instanciate cells that will be used in the game
        '''
        # Loop in X then Y and draw rectangles
        for grid_x in range(0, self.nbr_cell_width):
            x1 = grid_x * self.cellsize
            x2 = grid_x * self.cellsize + self.cellsize
            self.cells.append([])
            for grid_y in range(0, self.nbr_cell_height):
                y1 = grid_y * self.cellsize
                y2 = grid_y * self.cellsize + self.cellsize
                # Append the cell in the double list array
                self.cells[grid_x].append(Cell(grid_x, grid_y,
                                               x1, y1, x2, y2,
                                               self.canvas))
    
    def swap_status(self, tkevent):
        '''
        Change the status of a cell
        Called from a click
        '''
        # Get the position on the grid from the tkinter coordinates
        x, y = self._xy_to_grid_idx(tkevent.x, tkevent.y)
        
        # Change the current status of the cell and draw it
        self.cells[x][y].isActive = not self.cells[x][y].isActive
        self.cells[x][y].draw()
    
    def actualize_grid():
        '''
        Update the cell.isActivate attribute
        and display the grid using the updated value
        '''
        for x, row in enumerate(self.cells):
            for y, cell in enumerate(row):
                self.cells[x][y].isActive = self.cells[x][y].nextGen
                self.cells[x][y].draw()
                
    def calculate_next_gen(self, cell):
        '''
        Calculate the next value of the cell, either
        True (alive)
        or False (dead)
        '''
        cell_x = cell.grid_x
        cell_y = cell.grid_y
        nbr_neighbors = self._get_alive_neighbors(cell)
        
        # Rules of the game
        # Any live cell with two or three neighbors survives
        if nbr_neighbors in [2, 3]:
            self.cells[x][y].nextGen = True
        # Any dead cell with three live neighbors becomes a live cell
        elif self.cells[x][y].isAlive and nbr_neighbors == 3:
            self.cells[x][y].nextGen = True
        # All other live cells die in the next generation
        else:
            self.cells[x][y].nextGen = False
        
        
        
    def _get_alive_neighbors(self, cell):
        '''
        Return the number of cells alives in the vicinity of the given cell
        '''
        # TODO
        cell_x = cell.grid_x
        cell_y = cell.grid_y
        
        pass
    
    def _xy_to_grid_idx(self, x, y):
        '''
        Translate a x and y tkinter coordinates in a grid xy position
        Trick : I use int() to round down my coordinates
        '''
        return(int(x / self.cellsize),
               int(y / self.cellsize))
    
    def _debug(self):
        '''
        Print all cells and their status
        '''
        for row in self.cells:
            for cell in row:
                print(cell)


class Cell():
    '''
    This class represent a cell as intended in the game of life
    '''
    def __init__(self, grid_x, grid_y, x1, y1, x2, y2, canvas):
        
        # Position of the cell in the grid
        self.grid_x = grid_x
        self.grid_y = grid_y
        
        # Postition of the cell in the canvas
        self.x1 = x1
        self.x2 = x2
        self.y1 = y1
        self.y2 = y2
        
        # The canvas used to perform graphical magic
        self.canvas = canvas
        
        # Used for the game logic
        self.isActive = False
        self.nextGen = False
        
        # Automatically display the cell when instanciate
        self.draw()
        
        
    def draw(self):
        '''Draw the cell'''
        
        color = ''
        
        # Cell is alive
        if self.isActive == False:
            color = 'white'
        # Cell is dead
        elif self.isActive == True:
            color = 'black'
        # Draw the rectangle
        self.canvas.create_rectangle(self.x1, self.y1,
                                     self.x2, self.y2,
                                     fill=color)

    def __str__(self):
        return '{}, {} : {} --> {}'.format(self.grid_x, self.grid_y,
                                           self.isActive, self.nextGen)
         
class Game():
    '''
    This class contains the game logic
    '''
    def __init__(self):
        # Game default values
        width=720
        height=720
        cellsize=20
        
        # Instanciation of the main windows
        self.root = tk.Tk()
        self.root.title("My game of life ! \0/")
        
        # Instanciation of the frame on which the canvas will be added
        # pack() organizes widgets in blocks before placing them in the parent widget
        # Without it, the main windows will remain at default size
        # https://www.tutorialspoint.com/python/tk_pack.htm
        self.frame = tk.Frame(self.root, width=width, height=height)
        self.frame.pack()

        # Instanciation of the Canvas
        # The Canvas widget provides structured graphics facilities for Tkinter
        self.canvas = tk.Canvas(self.frame, width=width, height=height)
        self.canvas.pack()
    
        # Place buttons and link functions to them
        start_button = tk.Button(self.root, text="Start game", command=self.start)
        start_button.pack(side = tk.LEFT)
        stop_button = tk.Button(self.root, text="Stop it", command=self.stop)
        stop_button.pack(side = tk.RIGHT)
    
        # Create the grid and generate the visible rectangles
        self.grid = Grid(self.canvas, width, height, cellsize)
        self.grid.generate_grid()
        
        # link the left click action to the swap status function
        self.canvas.bind("<Button-1>", self.grid.swap_status)
        
        self.root.mainloop()
        
    def start(self):
        '''Start the game'''
        # I don't want a petit rigolo to change the status of cells
        self.canvas.unbind("<Button-1>")
        
        for x, row in enumerate(self.grid.cells):
            for y, cell in enumerate(row):
                self.grid.calculate_next_gen(self.grid.cells[x][y])
                
                    

    def stop(self):
        self.root.destroy()

game = Game()
# game.grid._debug()


SyntaxError: invalid syntax (<ipython-input-75-a47291e813f2>, line 69)